In [21]:
import struct
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

path = 'data/'

def load_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        # Read the magic number and metadata
        magic, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        if magic != 2051:  # Magic number for image files
            raise ValueError(f"Invalid magic number {magic} in file: {file_path}")
        
        # Read image data and reshape
        image_data = np.frombuffer(f.read(), dtype=np.uint8)
        images = image_data.reshape((num_images, rows, cols))
        return images / 255.0  # Normalize pixel values to [0, 1]

def load_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        # Read the magic number and metadata
        magic, num_labels = struct.unpack('>II', f.read(8))
        if magic != 2049:  # Magic number for label files
            raise ValueError(f"Invalid magic number {magic} in file: {file_path}")
        
        # Read label data
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

train_images = load_mnist_images(path + 'train-images.idx3-ubyte')
train_labels = load_mnist_labels(path + 'train-labels.idx1-ubyte')
test_images = load_mnist_images(path + 't10k-images.idx3-ubyte')
test_labels = load_mnist_labels(path + 't10k-labels.idx1-ubyte')

In [22]:
x_train = train_images.reshape(-1, 28*28)
x_test = test_images.reshape(-1, 28*28)

# Autoencoder architecture
encoding_dim = 16  # Size of encoded representation
input_img = tf.keras.layers.Input(shape=(784,))
encoded = tf.keras.layers.Dense(encoding_dim, activation='relu')(input_img)
decoded = tf.keras.layers.Dense(784, activation='sigmoid')(encoded)

# Build autoencoder
encoder = tf.keras.models.Model(input_img, encoded)
autoencoder = tf.keras.models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train autoencoder
autoencoder.fit(x_train, x_train, epochs=10, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4173 - val_loss: 0.2198
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2081 - val_loss: 0.1816
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1779 - val_loss: 0.1626
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1608 - val_loss: 0.1516
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1512 - val_loss: 0.1455
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1459 - val_loss: 0.1419
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1429 - val_loss: 0.1396
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1405 - val_loss: 0.1373
Epoch 9/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1381 - val_loss: 0.1355
Epoch 10/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1367 - val_loss: 0.1340


In [23]:
# Define the encoder model
encoder = tf.keras.models.Model(input_img, encoded)

# Define the decoder model
encoded_input = tf.keras.layers.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]  # Get the decoder layer
decoder = tf.keras.models.Model(encoded_input, decoder_layer(encoded_input))

In [24]:
# Encode an image
train_x = encoder.predict(x_train)  # Encode the first test image
test_x = encoder.predict(x_test)  # Encode the first test image

train_y = train_labels
test_y = test_labels

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 196us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 200us/step


In [25]:
print(train_x[0])

[ 3.3321717  4.410624   0.        10.545436   5.736751   7.9648137
  7.332858  15.161141   2.6526465 10.958398   9.660989   9.681724
 16.422173   7.306775   9.729878   7.9404435]


In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(16,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
          loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)

model.fit(train_x, train_y, epochs=10)

test_loss, test_acc = model.evaluate(test_x, test_y)
print(f"Test accuracy: {test_acc}")

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 280us/step - accuracy: 0.7532 - loss: 0.8973
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 268us/step - accuracy: 0.9136 - loss: 0.2766
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step - accuracy: 0.9338 - loss: 0.2197
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 266us/step - accuracy: 0.9407 - loss: 0.1952
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 260us/step - accuracy: 0.9436 - loss: 0.1830
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 266us/step - accuracy: 0.9481 - loss: 0.1662
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 281us/step - accuracy: 0.9512 - loss: 0.1561
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 265us/step - accuracy: 0.9517 - loss: 0.1493
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step - accuracy: 0.9540 - loss: 0.1437
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 278us/step - accuracy: 0.9556 - loss: 0.1429
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step - accuracy: 0.9474 - loss: 0.1673
Test accuracy: 0.95609